In [ ]:
import numpy as np
import pandas as pd
import optuna
import dagshub
import mlflow
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import OrdinalEncoder, TargetEncoder
from sklearn.svm import SVR
from feature_engine.encoding import MeanEncoder
from category_encoders import BinaryEncoder
from feature_engine.encoding import MeanEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
train = pd.read_parquet('C:/Users/aksha/OneDrive/Desktop/AutoNexusMlOps/data/Exp/train.parquet')
xtrain = train.drop(columns=['Price'])
ytrain = train['Price'].copy()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
xtrain.sample()

In [ ]:
def objective(trial):
    encoder_type = trial.suggest_categorical("encoder_type", ["Binary", "Target"])

    if encoder_type == "Binary":
        encoder = ColumnTransformer([
            ('ordinal_encoding',
             OrdinalEncoder(categories=[['New', 'Certified', 'Used']]),
             ['Stock_Type']),
            ('BinaryEncoder',
             BinaryEncoder(),
             ['Brand_Name', 'Model_Name', 'Exterior_Color',
              'Interior_Color', 'Drivetrain', 'Fuel_Type',
              'Cylinder_Config', 'City', 'STATE'])
        ],remainder='passthrough')
    
    else:
        encoder = ColumnTransformer([
            ('ordinal_encoding',
             OrdinalEncoder(categories=[['New', 'Certified', 'Used']]),
             ['Stock_Type']),
            ('TargetEncoder',
             TargetEncoder(),
             ['Brand_Name', 'Model_Name', 'Exterior_Color',
              'Interior_Color', 'Drivetrain', 'Fuel_Type',
              'Cylinder_Config', 'City', 'STATE'])
        ], remainder='passthrough')

    model_type = trial.suggest_categorical("model", ["Linear", "Ridge", "Lasso", "ElasticNet","SVR"])

    if model_type == "Linear":
        model = LinearRegression()

    elif model_type == "Ridge":
        model = Ridge(
            alpha=trial.suggest_float("ridge_alpha", 1e-3, 10.0, log=True),
            solver=trial.suggest_categorical("ridge_solver",  ["svd", "cholesky", "lsqr"])
        )

    elif model_type == "Lasso":
        model = Lasso(
            alpha=trial.suggest_float("lasso_alpha", 1e-3, 10.0, log=True),
            max_iter=trial.suggest_int("lasso_max_iter", 1000, 10000),
            tol=trial.suggest_float("lasso_tol", 1e-5, 1e-2, log=True),
            selection=trial.suggest_categorical("lasso_selection", ["cyclic", "random"])
        )
    
    elif model_type == "SVR":
        model = SVR(
          C=trial.suggest_float("svr_C", 1e-2, 100.0, log=True),
          kernel=trial.suggest_categorical("svr_kernel", ["linear", "poly", "rbf", "sigmoid"]),
          degree=trial.suggest_int("svr_degree", 2, 5),
          gamma=trial.suggest_categorical("svr_gamma", ["scale", "auto"]),
          epsilon=trial.suggest_float("svr_epsilon", 1e-3, 1.0, log=True),
          tol=trial.suggest_float("svr_tol", 1e-5, 1e-2, log=True),
          max_iter=trial.suggest_int("svr_max_iter", 1000, 10000)
    )

    else:  
        model = ElasticNet(
            alpha=trial.suggest_float("elastic_alpha", 1e-3, 10.0, log=True),
            l1_ratio=trial.suggest_float("elastic_l1_ratio", 0.0, 1.0),
            max_iter=trial.suggest_int("elastic_max_iter", 1000, 10000),
            tol=trial.suggest_float("elastic_tol", 1e-5, 1e-2, log=True),
            selection=trial.suggest_categorical("elastic_selection", ["cyclic", "random"])
        )
    
    scaler = ColumnTransformer(
        [
            ('std', StandardScaler(), ['Km/L']),
            ('norm', MinMaxScaler(), ['Model_Year','Valves']),
            ('robust', RobustScaler(),['Mileage']),
            ('maxabs', MaxAbsScaler(), ['Level2_Charging','Dc_Fast_Charging','Battery_Capacity','Gear_Spec','Km/L_e_City','Km/L_e_Hwy'])
        ],
        remainder='passthrough'
    )
    
    cols_for_encoding = ['Brand_Name', 'Model_Name', 'Exterior_Color',
                'Interior_Color', 'Drivetrain', 'Fuel_Type',
                'Cylinder_Config', 'City', 'STATE', 'Stock_Type']
    
    cols_for_scaling = ['Model_Year','Valves','Km/L','Mileage','Level2_Charging','Dc_Fast_Charging','Battery_Capacity','Gear_Spec','Km/L_e_City','Km/L_e_Hwy']
    
    transformer = ColumnTransformer([
        ('encoder',encoder,cols_for_encoding),
        ('scaler', scaler,cols_for_scaling),
    ])

    pipe = Pipeline(
        [
            ('transformer',transformer),
            ('model',model)
        ]
    )

    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(pipe, xtrain, ytrain, cv=cv, scoring='r2', n_jobs=-1).mean()

    return score

In [ ]:
mlflow.set_tracking_uri('https://dagshub.com/akshatsharma2407/AutoNexusMlOps.mlflow')
dagshub.init(repo_owner='akshatsharma2407', repo_name='AutoNexusMlOps', mlflow=True)

In [ ]:
study = optuna.create_study(
      study_name='Linear Algos with Different Encoding & Scaling techniques',
      direction='maximize',
      storage='sqlite:///../reports/autonexus_optuna.db',
      load_if_exists=True
    )

study.optimize(objective,n_trials=30)

In [ ]:
mlflow.set_experiment(experiment_name='Linear Algos with Different Encoding & Scaling techniques')
for trial in study.trials:
    with mlflow.start_run(run_name=f"trial_{trial.number}"):
        mlflow.log_params(trial.params)
        
        if trial.value:
            mlflow.log_metric("objective", trial.value)
        
        mlflow.set_tag("state", trial.state.name)
        mlflow.set_tag("trial_number", trial.number)